quotes_to_scrape

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup

In [2]:
def clean_string(s: str) -> str:
    return ''.join(filter(str.isdigit, s))

In [3]:
url = "https://gazovikperm.ru/katalog.html"
url_base = "https://gazovikperm.ru/"
r = requests.post(url)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'lxml')

In [4]:
res_mas = []
caterory_inner_images = soup.find('div', class_="caterory_inner_images")
hrefs = caterory_inner_images.find_all('a')
for href in hrefs:
    # print(href['href'])
    ref1 = url_base + href['href'].replace('\n', '')
    r = requests.post(ref1)
    html_doc = r.text
    soup1 = BeautifulSoup(html_doc, 'lxml')
    groups_1 = soup1.find('div', class_="caterory_inner_images") # Каталог 2 порядка
    hrefs1 = groups_1.find_all('a')
    
    for href1 in hrefs1:
        try:
            cur_len = 0
            ref2 = url_base + href1['href'].replace('\n', '') + f"&offset={cur_len}"
            # print(ref2) 
            r = requests.post(ref2)
            html_doc = r.text
            soup2 = BeautifulSoup(html_doc, 'lxml')

            name = soup2.find('main', class_="content_in content_in_semiwide").find("h1").text

            len_max_ = soup2.find('p', class_="col col-10")
            if len_max_ == None:
                continue
            len_max = int(clean_string(len_max_.text))
            cur_len = 0

            while cur_len < len_max:
                ref2 = url_base + href1['href'].replace('\n', '') + f"&offset={cur_len}"
                r = requests.post(ref2)
                html_doc = r.text
                soup2 = BeautifulSoup(html_doc, 'lxml')
                items = soup2.find_all('div', class_= "list-items")
                cur_len += len(items)
                for item in items:
                    # print(ref2)

                    cost_ = item.find('span', class_="js-price-value")
                    if cost_ == None:
                        cost_ = item.find('span', class_="js-item-disabled")
                        if cost_ == None:
                            cost = ""
                        else:
                            cost = cost_.text
                    else:
                        cost = cost_.text
                    title = item.find('a', class_="img")['title']
                    desc = item.find('div', class_="item-properties")
                    descs = desc.find_all('p')
                    desc_dict = {}
                    for p in descs:
                        desc_dict[p.find('span', class_="l1").text] = p.find('span', class_="l2").text
                    # print(desc_dict)
                    res_mas.append([title, cost, desc_dict])
            
            df = pd.DataFrame(res_mas, columns= ['title', 'cost', 'desc_dict'])
            df.to_csv(
                f'./res_data_1/{name}.csv',
                index=False,           # не записывать индекс
                sep=',',              # разделитель
                encoding='utf-8',     # кодировка
                header=True,          # записать заголовки
                na_rep='NULL'         # представление для NaN значений
            )

        except Exception as e:
            print(ref2)
            raise e


        
    